# GPT Model w/ Descriptions
Combined_Songs_Artists.csv

Model trained on Playlist_Names, Playlist_Descriptions, Songs and Artists combined in one string

In [7]:
import pandas as pd
from datasets import Dataset
import ast
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
import os
import csv
import optuna
import torch

In [2]:
os.chdir('..')
os.getcwd()

'/sfs/gpfs/tardis/home/nuf8ms/Documents/MSDS/LLM'

In [2]:
#df2 = pd.read_csv('DS6051-Project/Playlist_data_with_lyrics.csv')
#df3 = pd.read_csv('DS6051-Project/playlist_data.csv')
df = pd.read_csv('../Additional Data/Combined_Songs_Artists.csv')

In [3]:
# Load and parse the dataset

# Convert stringified lists to actual lists
# for col in ['Playlist_Songs', 'Playlist_Artists']:
#     df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

df["Playlist_Songs"] = df["Playlist_Songs"].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

## Training Targets 1st Iteration, 3 epoch Loss ~ 3, 5 epoch Loss ~ 2.5
## playlist_data.csv
# Create prompt + output text for GPT-2 training
# def format_example(row):
#     song_lines = [f"{s} - {a}" for s, a in zip(row['Playlist_Songs'], row['Playlist_Artists'])]
#     return f"### Prompt: {row['Playlist_Name']}\n### Playlist:\n" + "\n".join(song_lines)

## Training Targets 2nd Iteration, 5 epoch Loss ~ 1.3
## playlist_data.csv
# def format_example(row):
#     lines = [f"[SONG] {s} [ARTIST] {a}" for s, a in zip(row["Playlist_Songs"], row["Playlist_Artists"])]
#     return f"### Prompt: {row['Playlist_Name']}\n### Playlist:\n" + "\n".join(lines)

## Training targets 3rd Iteration, 5 epoch Loss ~ 1.35
## playlist_data.csv
# def format_example(row):
#     lines = [f"[SONG] {song} [ARTIST] {artist}" for song, artist in zip(row["Playlist_Songs"], row["Playlist_Artists"])]
#     playlist_body = "\n".join(lines)
    
#     # Include Playlist_Description in the prompt for training only
#     return (
#         f"### Prompt: {row['Playlist_Name']}\n"
#         f"### Description: {row['Playlist_Description']}\n"
#         f"### Playlist:\n{playlist_body}"
#     )

## Training targets 4th Iteration, 
## Combined_Songs_Artists.csv
def format_example(row):
    # Use the song-artist strings as-is
    playlist_body = "\n".join(row["Playlist_Songs"])
    return (
        f"### Prompt: {row['Playlist_Name']}\n"
        f"### Description: {row['Playlist_Description']}\n"
        f"### Playlist:\n{playlist_body}"
    )

df['text'] = df.apply(format_example, axis=1)

# Convert to Hugging Face Dataset
dataset = Dataset.from_dict({"text": df['text'].tolist()})

In [4]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

tokenizer = GPT2Tokenizer.from_pretrained("gpt2-medium")
tokenizer.pad_token = tokenizer.eos_token  # GPT-2 doesn't have a pad token

model = GPT2LMHeadModel.from_pretrained("gpt2-medium")
model.resize_token_embeddings(len(tokenizer))  # In case we add special tokens

# Tokenize
def tokenize(batch):
    encodings = tokenizer(batch["text"], padding="max_length", truncation=True, max_length=512)
    encodings["labels"] = encodings["input_ids"].copy()  # 🔥 Add this line
    return encodings
    
tokenized_dataset = dataset.map(tokenize, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

Map: 100%|██████████| 2289/2289 [00:07<00:00, 310.51 examples/s]


### Test Script

### Test Script

In [11]:
training_args = TrainingArguments(
    output_dir="./models/gpt2_Combined_Song_Artists_Checkpoints",
    overwrite_output_dir=True,
    num_train_epochs=30,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_dir="./logs",
    logging_steps=50,
    fp16=True if torch.cuda.is_available() else False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset
)

trainer.train()

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
50,3.371100
100,2.917400
150,2.865700
200,2.789100
250,2.698200
300,2.682200
350,2.684800
400,2.548800
450,2.555700
500,2.565100


TrainOutput(global_step=34350, training_loss=0.9339163249097711, metrics={'train_runtime': 5592.7868, 'train_samples_per_second': 12.278, 'train_steps_per_second': 6.142, 'total_flos': 6.377387669323776e+16, 'train_loss': 0.9339163249097711, 'epoch': 30.0})

In [12]:
model.save_pretrained("./models/gpt2_Combined_Song_Artists")
tokenizer.save_pretrained("./models/gpt2_Combined_Song_Artists")

('./models/gpt2_Combined_Song_Artists/tokenizer_config.json',
 './models/gpt2_Combined_Song_Artists/special_tokens_map.json',
 './models/gpt2_Combined_Song_Artists/vocab.json',
 './models/gpt2_Combined_Song_Artists/merges.txt',
 './models/gpt2_Combined_Song_Artists/added_tokens.json')

In [1]:
def generate_playlist(prompt, max_length=200):
    input_text = f"### Prompt: {prompt}\n### Playlist:\n"
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output = model.generate(
        input_ids = input_ids.to('cuda'),
        max_length=max_length,
        temperature=0.9,
        top_p=0.95,
        do_sample=True,
        num_return_sequences=1,
        pad_token_id=tokenizer.eos_token_id
    )

    result = tokenizer.decode(output[0], skip_special_tokens=True)
    return result.split("### Playlist:\n")[1].strip()

# Try it out!
prompt = "Hype Pregame"
print(generate_playlist(prompt))

NameError: name 'tokenizer' is not defined